# Modeling the Orderbook for Probabilistic Prediction

[!] Define below setups, use recommended hyperparameters

In [11]:
from OrderFusion_Modeling import *

save_path = train_status('local') # 'local' if run locally or 'cloud' run on Google Colab
countries = ['germany'] # countries, 'austria' or 'germany'
resolutions = ['h'] # data resolution, 'h' for hourly, 'qh' for quarter-hourly
indices = ['ID3'] # target indices, 'ID3', 'ID2', or 'ID1'

train_start_date = '2022-01-01' # start date for training data
split_len = (24, 6, 6) # 24 months for training, 6 months for validation, 6 months for testing
quantiles = [0.1, 0.25, 0.45, 0.5, 0.55, 0.75, 0.9] # target quantiles to be predicted

seeds = [42] # random seeds, default is 1
model_shapes = [[64, 2]] # recommended shapes is [64, 2] (hidden_dimension and number of blocks)
epoch, batch_size = 50, 1024 # recommended number of epochs and batch size are 50 and 1024 respectively

points = [4] # T_max / number of trades in the input sequence
''' recommended values for T_max are:
    64 for ID1 and ID2, and 4 for ID3
'''

decay_strength = 2 # decay strength for the binary masking
''' recommended values for decay_strength are:
    6 for ID1 and ID2, and 2 for ID3
'''

target_model = 'OrderFusion'

# configuration
data_config = (countries, resolutions, indices, save_path, split_len, train_start_date)
model_config = (target_model, model_shapes, epoch, batch_size, points, quantiles, seeds, decay_strength)

After defining above setups, just by runing the below block

Note that:
- This produces the model called 'OrderFusion.keras' in the 'Model' subfolder
- different random seed introduces slightly variance
- Please use the recommended hyperparameters from the previous block


In [12]:
# execute experiments
execute_main(data_config, model_config)

('germany', 'h', 'ID3') | point=4 | OrderFusion | model_shape: [64, 2] | seed: 42
paras: 162183
138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step
AQL: 3.73, AQCR: 0.0, RMSE: 24.12, MAE: 8.94, R2: 0.88, Inference time: 3.14043927192688s 



You can load the trained model by runnng the below block

In [5]:
custom_objects = {f'{name}_label': quantile_loss(q, name) for name, q in {f'q{q:02}': q / 100 for q in [int(q * 100) for q in quantiles]}.items()}
checkpoint_path = os.path.join(f"{save_path}Model", f"{target_model}.keras")

with custom_object_scope(custom_objects):
    best_model = load_model(checkpoint_path, custom_objects=custom_objects)
    print(f"{target_model} model successfully loaded")

OrderFusion model successfully loaded
